In [1]:
import os
import pickle
import random
import glob
import datetime
import pandas as pd
import numpy as np
import cv2
import pydicom
from tqdm import tqdm
from joblib import delayed, Parallel
import zipfile
from pydicom.filebase import DicomBytesIO
import sys
from PIL import Image
import cv2
#from focal_loss import sparse_categorical_focal_loss
import keras
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.applications.inception_v3 import InceptionV3

# importing pyplot and image from matplotlib 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg 
from keras.optimizers import SGD
from keras import backend
from keras.models import load_model

from keras.preprocessing import image
import albumentations as A


from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import tensorflow as tf

Using TensorFlow backend.


In [2]:
from tensorflow.keras.losses import Reduction

from tensorflow_addons.losses import SigmoidFocalCrossEntropy

In [3]:
base_url = '/home/ubuntu/kaggle/rsna-intracranial-hemorrhage-detection/'
TRAIN_DIR = '/home/ubuntu/kaggle/rsna-intracranial-hemorrhage-detection/stage_2_train/'
TEST_DIR = '/home/ubuntu/kaggle/rsna-intracranial-hemorrhage-detection/stage_2_test/'
image_dir = '/home/ubuntu/kaggle/rsna-intracranial-hemorrhage-detection/png/train/adjacent-brain-cropped/'
save_dir = 'home/ubuntu/kaggle/models/'
os.listdir(base_url)

def png(image): 
    return image + '.png'

# Learning rate scheduler

In [5]:
initial_learning_rate = 1e-2
first_decay_steps = 1000
lr_decayed_fn = (
  tf.keras.experimental.CosineDecayRestarts(
      initial_learning_rate,
      first_decay_steps))
opt = tf.keras.optimizers.Adam(learning_rate=lr_decayed_fn)

# Weighted_metrics

In [6]:
from keras import backend as K
def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return K.mean(loss_samples)



In [7]:
train_idg = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,
        rescale=1./255)
valid_idg = ImageDataGenerator(rescale=1./255)
training_data = pd.read_csv(f'train_1.csv') 
training_data['Image'] = training_data['Image'].apply(png)

validation_data = pd.read_csv(f'valid_1.csv')
validation_data['Image'] = validation_data['Image'].apply(png)

columns=['any','epidural','intraparenchymal','intraventricular', 'subarachnoid','subdural']

train_data_generator = train_idg.flow_from_dataframe(training_data, directory = image_dir,
                           x_col = "Image", y_col = columns,batch_size=64,
                           class_mode="raw", target_size=(224,224), shuffle = True)
valid_data_generator  = valid_idg.flow_from_dataframe(validation_data, directory = image_dir,
                        x_col = "Image", y_col = columns,batch_size=64,
                        class_mode = "raw",target_size=(224,224), shuffle = False)

Found 604739 validated image filenames.
Found 148063 validated image filenames.


In [8]:
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc')
      
]







# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(256, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
x = Dropout(0.3)(x)
#x = Dense(100, activation="relu")(x)
#x = Dropout(0.3)(x)
#pred = Dense(6,
#             kernel_initializer=tf.keras.initializers.HeNormal(seed=11),
#             kernel_regularizer=tf.keras.regularizers.l2(0.05),
#             bias_regularizer=tf.keras.regularizers.l2(0.05), activation="softmax")(x)
#initializer = keras.initializers.GlorotUniform()
#layer = tf.keras.layers.Dense(3, kernel_initializer=initializer)

predictions = Dense(6, activation='softmax')(x)
#activation='sigmoid',kernel_initializer=keras.initializers.GlorotNormal()
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(opt, loss=SigmoidFocalCrossEntropy(), metrics= METRICS)
#model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=METRICS)
#model.compile(loss=loss_func,
#          optimizer=opt,
#          metrics=METRICS)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

# callbacks

In [9]:
from keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint


checkpoint = tf.keras.callbacks.ModelCheckpoint('resnet50{epoch:08d}.h5', period=1,mode= 'auto',save_best_only=True) 

learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_acc', 
                                            patience=1, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callback_list = [checkpoint]


In [10]:
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


# model fit

In [11]:



num_epochs = 3

batch_size = 128
training_steps = len(training_data) // batch_size
validation_step = len(validation_data) // batch_size




# FIT THE MODEL
history = model.fit(train_data_generator,
            epochs=num_epochs,steps_per_epoch=training_steps,
            callbacks=callback_list,
            validation_data=valid_data_generator,
            validation_steps= validation_step) 





tf.keras.backend.clear_session()



/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/3
4724/4724 [==============================] - 5855s 1s/step - loss: 0.6288 - tp: 6640.0000 - fp: 51458.0000 - tn: 1658587.0000 - fn: 96965.0000 - accuracy: 0.9182 - precision: 0.1143 - recall: 0.0641 - auc: 0.5777 - val_loss: 0.6626 - val_tp: 1.0000 - val_fp: 30.0000 - val_tn: 419245.0000 - val_fn: 24628.0000 - val_accuracy: 0.9445 - val_precision: 0.0323 - val_recall: 4.0603e-05 - val_auc: 0.6240
Epoch 2/3
4724/4724 [==============================] - 5560s 1s/step - loss: 0.7789 - tp: 17593.0000 - fp: 138643.0000 - tn: 1572177.0000 - fn: 85237.0000 - accuracy: 0.8766 - precision: 0.1126 - recall: 0.1711 - auc: 0.6140 - val_loss: 0.8053 - val_tp: 8792.0000 - val_fp: 65190.0000 - val_tn: 354085.0000 - val_fn: 15837.0000 - val_accuracy: 0.8175 - val_precision: 0.1188 - val_recall: 0.3570 - val_auc: 0.6379
Epoch 3/3
4724/4724 [==============================] - 5500s 1s/step - loss: 0.8205 - tp: 38155.0000 - fp: 264178.0000 - tn: 1446569.0000 - fn: 65114.0000 - accuracy: 0.8185 - 

In [12]:
tf.keras.backend.clear_session()
#from keras.models import load_model

In [12]:
model = tf.keras.models.load_model('resnet5000000001.h5')

In [12]:
X_val_sample, val_labels = next(valid_data_generator)
predictions = model.predict_generator(valid_data_generator,steps = 128)

Instructions for updating:
Please use Model.predict, which supports generators.


In [ ]:
labels = ['any','epidural','intraparenchymal','intraventricular', 'subarachnoid','subdural']
cm = confusion_matrix(validation_data['any','epidural','intraparenchymal','intraventricular', 'subarachnoid','subdural'], prediction_, labels)
print(cm)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the classifier')
fig.colorbar(cax)
ax.set_xticklabels([''] + labels)
ax.set_yticklabels([''] + labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
BATCH_SIZE = 64
test_predictions_weighted = model.predict(valid_data_generator, batch_size=BATCH_SIZE)


weighted_results = model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, weighted_results):
    print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

In [15]:
valid_predict =  model.evaluate_generator(valid_data_generator)
print(valid_predict)

[0.6691614985466003, 1.0, 89.0, 838395.0, 49893.0, 0.9437377452850342, 0.011111111380159855, 2.004248926823493e-05, 0.6232836246490479]


In [13]:
model.metrics_names

['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']

In [ ]:
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(valid_predict[2], valid_predict[1], marker='.')
# show the plot
plt.show()

In [17]:
def plot_training(H):
    # construct a plot that plots and saves the training history
    with plt.xkcd():
        plt.figure(figsize = (10,10))
        plt.plot(H.epoch,H.history["acc"], label="train_acc")
        plt.plot(H.epoch,H.history["val_acc"], label="val_acc")
        plt.title("Training Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Accuracy")
        plt.legend(loc="lower left")
        plt.show()

In [ ]:
plot_training(model)

In [ ]:
def plot_training(H):
    # construct a plot that plots and saves the training history
    with plt.xkcd():
        plt.figure(figsize = (10,10))
        plt.plot(H.epoch,H.history["loss"], label="train_loss")
        plt.plot(H.epoch,H.history["val_loss"], label="val_loss")
        plt.title("Training Loss")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss")
        plt.legend(loc="lower left")
        plt.show()

In [ ]:
plot_training(history)

In [ ]:
def plot_metrics(history):
    plt.figure(figsize = (15,15))
    metrics =  ['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
    for n, metric in enumerate(metrics):
        
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        
        plt.plot(history.epoch,  history.history[metric], color='red', label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
                 color='orange', linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.8,1])
        else:
            plt.ylim([0,1])

        plt.legend()

In [ ]:
plot_training(history)